In [15]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from tkinter.font import Font
import psycopg2
from bs4 import BeautifulSoup
import bs4
import os
from tkinterdnd2 import TkinterDnD, DND_FILES
import re
import datetime

def parse_date_string(date_string):

    date_string = date_string.lower()  # Для упрощения сравнения

    if "сегодня" in date_string:
        return datetime.date.today()
    elif "вчера" in date_string:
        return datetime.date.today() - datetime.timedelta(days=1)
    else:
        # Пытаемся обработать формат "ДД месяц ГГГГ"
        match = re.search(r"был (\d{1,2})\xa0(.*?)\xa0(\d{4})", date_string)
        if match:
            day = int(match.group(1))
            month_name = match.group(2).strip()
            year = int(match.group(3))

            month_dict = {
                "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
                "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,
            }

            if month_name in month_dict:
                month = month_dict[month_name]
                try:
                    return datetime.date(year, month, day)
                except ValueError:
                    return None

        else:
            # Пытаемся обработать формат "ДД месяц"
            match = re.search(r"был (\d{1,2})\xa0(.*)", date_string)
            if match:
                day = int(match.group(1))
                month_name = match.group(2).strip()

                month_dict = {
                    "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
                    "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,
                }

                if month_name in month_dict:
                    month = month_dict[month_name]
                    year = datetime.date.today().year
                    try:
                        return datetime.date(year, month, day)
                    except ValueError:
                        return None

    return None  # Если не удалось распознать дату


class DnDFrame(ttk.LabelFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.drop_target_register(DND_FILES)
        self.dnd_bind('<<Drop>>', self.handle_drop)
        
        self.file_listbox = tk.Listbox(self, height=5)
        self.file_listbox.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        btn_frame = ttk.Frame(self)
        btn_frame.pack(fill=tk.X, pady=5)
        
        self.clear_btn = ttk.Button(btn_frame, text="Очистить", command=self.clear_files)
        self.clear_btn.pack(side=tk.LEFT, padx=5)
        
        self.add_btn = ttk.Button(btn_frame, text="Добавить файлы", command=self.add_files)
        self.add_btn.pack(side=tk.RIGHT, padx=5)
    
    def handle_drop(self, event):
        files = self.tk.splitlist(event.data)
        for file in files:
            if file.lower().endswith('.html'):
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
            else:
                messagebox.showwarning("Неверный формат", f"Файл {file} не является HTML")
    
    def add_files(self):
        filetypes = (('HTML files', '*.html'), ('All files', '*.*'))
        files = filedialog.askopenfilenames(title='Выберите файлы', filetypes=filetypes)
        if files:
            for file in files:
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
    
    def clear_files(self):
        self.file_listbox.delete(0, tk.END)
    
    def get_files(self):
        return list(self.file_listbox.get(0, tk.END))

class Application(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()
        self.title("Парсер вакансий HH")
        self.geometry("600x600")  # Увеличили высоту окна
        self.configure(bg='#f0f0f0')
        
        self.DB_CONFIG = {
            'host': 'localhost',
            'port': '5433',
            'database': 'hh_test',
            'user': 'postgres',
            'password': '123'
        }
        
        self.current_comment = ""  # Текущий комментарий для сессии парсинга
        self.create_widgets()
    
    def create_widgets(self):
        # Стилизация
        bold_font = Font(family="Arial", size=10, weight="bold")
        normal_font = Font(family="Arial", size=10)
        
        # Главный фрейм
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Заголовок
        ttk.Label(
            main_frame,
            text="Парсер вакансий HeadHunter",
            font=Font(family="Arial", size=12, weight="bold")
        ).pack(pady=(0, 10))
        
        # Область для комментария
        comment_frame = ttk.LabelFrame(main_frame, text="Комментарий к парсингу", padding=10)
        comment_frame.pack(fill=tk.X, pady=5, padx=5)
        
        self.comment_text = tk.Text(comment_frame, height=4, wrap=tk.WORD)
        self.comment_text.pack(fill=tk.BOTH, expand=True)
        
        ttk.Button(comment_frame, text="Сохранить комментарий", 
                 command=self.save_comment).pack(pady=5)
        
        # Область для перетаскивания файлов
        drag_frame = DnDFrame(main_frame, text="Перетащите HTML-файлы сюда или нажмите кнопку", 
                             width=500, height=150)
        drag_frame.pack(fill=tk.BOTH, pady=10, padx=10, expand=True)
        self.drag_frame = drag_frame
        
        # Кнопка обработки
        self.process_btn = ttk.Button(
            main_frame,
            text="Обработать файлы",
            command=self.process_files,
            style='Accent.TButton'
        )
        self.process_btn.pack(pady=10)
        
        # Статус бар
        self.status_var = tk.StringVar()
        status_bar = ttk.Label(
            main_frame,
            textvariable=self.status_var,
            relief=tk.SUNKEN,
            anchor=tk.W
        )
        status_bar.pack(fill=tk.X, pady=(10, 0))
        
        # Стиль
        style = ttk.Style()
        style.configure('Accent.TButton', font=bold_font, foreground='black')
    
    def save_comment(self):
        """Сохраняет комментарий для текущей сессии парсинга"""
        self.current_comment = self.comment_text.get("1.0", tk.END).strip()
        messagebox.showinfo("Сохранено", "Комментарий сохранен и будет добавлен ко всем обрабатываемым файлам")
    
    def process_files(self):
        files = self.drag_frame.get_files()
        if not files:
            messagebox.showwarning("Ошибка", "Нет файлов для обработки")
            return
        
        total_records = 0
        processed_files = 0
        
        self.process_btn.config(state=tk.DISABLED)
        self.status_var.set("Начата обработка файлов...")
        self.update()
        
        for file in files:
            try:
                data, metadata = self.parser(file)
                if self.load_db(data, metadata):
                    total_records += len(data)
                    processed_files += 1
                    self.status_var.set(f"Обработан {os.path.basename(file)} ({len(data)} записей)")
                    self.update()
                else:
                    messagebox.showerror("Ошибка", f"Ошибка при сохранении данных из {file}")
            except Exception as e:
                messagebox.showerror("Ошибка", f"Ошибка при обработке {file}: {str(e)}")
        
        self.status_var.set(f"Готово! Обработано {processed_files}/{len(files)} файлов, {total_records} записей")
        self.process_btn.config(state=tk.NORMAL)
        self.current_comment = ""  # Сбрасываем комментарий после обработки
    
    def parser(self, pathway):
        data = []
        with open(pathway, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            super_info = soup.find_all(class_='resume-card-content--pA9euQ2yPckXrBh1')
            if super_info:
                for load_vac in super_info:

                    start_time = datetime.datetime.now()
                    file_name = os.path.basename(pathway)
                    file_time_create = datetime.datetime.fromtimestamp(os.path.getctime(pathway))
                    type_file = 'Список резюме'

                    main_info = load_vac.find(class_='column-content--q3SfppwQANVUv38P')
                    name_resume = main_info.find('span', {'data-qa': 'serp-item__title-text'})
                    
                    if name_resume:
                        name_resume = name_resume.text
                    else:
                        name_resume = None
                    # Возраст
                    age = load_vac.find('span', {'data-qa': 'resume-serp__resume-age'})
                    if age:
                        age = age.text
                    else:
                        age = None

                    url_find = main_info.find('a')['href']
                    img_url = load_vac.find_all('img')
                    if img_url:
                        img_text = img_url[0]['src']
                        img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
                    else:
                        img_text = None  
        
                    info_about_time_to_be = main_info.find_all(class_='magritte-text___pbpft_4-1-1 magritte-text_style-secondary___1IU11_4-1-1 magritte-text_typography-label-3-regular___Nhtlp_4-1-1')
                
                    all_activity_dates = []
                    for element in info_about_time_to_be:
                        # Ищем все строки с "Был" или "Была"
                        activity_strings = element.find_all(string=lambda t: isinstance(t, str) and ('Был' in t or 'Была' in t))
                
                        for activity in activity_strings:
                            # Для каждой найденной строки находим следующую дату
                            date_span = activity.find_next('span')
                            if date_span:
                                full_text = f"{activity.strip()} {date_span.text.strip()}"
                                all_activity_dates.append(full_text)
                
                    # Если нашли активности, объединяем их через запятую
                    full_text = ', '.join(all_activity_dates) if all_activity_dates else None
                    full_text = parse_date_string(full_text)
                    short_url = re.match(r'^[^?]+', url_find).group()
        
                    data.append((name_resume, age, url_find, img_text, short_url, full_text))
        
                parsing_end = datetime.datetime.now()
                metadata = (file_name, file_time_create, start_time, parsing_end, type_file, self.current_comment)
                return data, metadata
            
            else:
                # Обработка отдельных резюме
                start_time = datetime.datetime.now()
                file_name = os.path.basename(pathway)
                file_time_create = datetime.datetime.fromtimestamp(os.path.getctime(pathway))
                type_file = 'Отдельное резюме'

                # HTML основная информация
                head = soup.find(class_ = 'header--FIqvP_vS2Y1E2k5a')

                name_resume = head.find('h2', {'data-qa': 'title'})
                if name_resume:
                    name_resume = name_resume.text
                else:
                    name_resume = None
                age = soup.find('span', {'data-qa': 'resume-personal-age'})
                if age:
                    age = age.text
                else:
                    age = None
                # URL
                comments = soup.find_all(string=lambda text: isinstance(text, bs4.Comment))
                url_find = None
                if comments:
                    comment_text = comments[0]
                    match = re.search(r'url=\((\d+)\)(.*)', comment_text)
                    if match:
                        url_find = match.group(2)
                        url_find = url_find.strip()
                
                img_url = soup.find_all('img', attrs={'alt': 'resume photo'})
                if img_url:
                    img_text = img_url[0]['src']
                    img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
                else:
                    img_text = None 
                
                short_url = re.match(r'^[^?]+', url_find).group()
                
                full_text = None
                data.append((name_resume, age, url_find, img_text, short_url, full_text))

                parsing_end = datetime.datetime.now()
                metadata = (file_name, file_time_create, start_time, parsing_end, type_file, self.current_comment)
                return data, metadata

    def load_db(self, data, metadata):
        try: 
            with psycopg2.connect(**self.DB_CONFIG) as connect:
                with connect.cursor() as cur:
                    # Создаем таблицы, если их нет
                    cur.execute('''CREATE TABLE IF NOT EXISTS info_res (
                            id SERIAL PRIMARY KEY,
                            session_id INTEGER,
                            name_resume TEXT,
                            age TEXT,
                            full_url TEXT,
                            img_text TEXT,
                            short_url TEXT,
                            visit_time DATE,
                            comment TEXT)''')
                    
                    cur.execute('''CREATE TABLE IF NOT EXISTS parsing_metadata (
                                    id SERIAL PRIMARY KEY,
                                    session_id INTEGER,
                                    html_file_name TEXT,
                                    file_creation_time TIMESTAMP,
                                    parsing_start_time TIMESTAMP,
                                    parsing_end_time TIMESTAMP,
                                    type_file TEXT,
                                    parsing_comment TEXT);''')
                    
                    # Получаем ID для текущей сессии
                    cur.execute("SELECT COALESCE(MAX(session_id), 0) FROM info_res")
                    max_id = cur.fetchone()[0] + 1
                    
                    # Подготавливаем данные для вставки
                    data_with_session = [(max_id, name_resume, age, url, img_text, short_url, visit_time, '') 
                                       for name_resume, age, url, img_text, short_url, visit_time in data]
                    
                    # Добавляем комментарий к метаданным
                    metadata_with_session = (max_id,) + metadata
                    
                    # Вставляем данные
                    cur.executemany('''INSERT INTO info_res 
                                    (session_id, name_resume, age, full_url, img_text, short_url, visit_time, comment) 
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''', 
                                 data_with_session)
                    
                    cur.execute('''INSERT INTO parsing_metadata 
                                (session_id, html_file_name, file_creation_time, 
                                 parsing_start_time, parsing_end_time, type_file, parsing_comment) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s)''', 
                             metadata_with_session)
                    
                    connect.commit()
                return True
            
        except Exception as e:
            print(f'Ошибка {e}')
            return False

if __name__ == "__main__":
    app = Application()
    app.mainloop()